In [20]:
import hopsworks
import joblib

project = hopsworks.login()

fs = project.get_feature_store()
mr = project.get_model_registry()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549021
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [21]:
import datetime
start_date = (datetime.datetime.now() - datetime.timedelta(days=7)) 
end_date = (datetime.datetime.now()) 
print(start_date)
print(end_date)

2024-04-30 08:18:04.627413
2024-05-07 08:18:04.627445


In [22]:
mr = project.get_model_registry()
model = mr.get_model("el3_random_forrest_model", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/el3_random_forrest_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.


In [27]:
feature_view = fs.get_feature_view(name="el_three_api", version=1)
#feature_view.init_serving()
#feature_view.init_batch_scoring()
#batch_data = feature_view.get_batch_data()

feature, label = feature_view.training_data(
    description = 'el3 training dataset',
)
feature = feature[['x','y','z']]

Finished: Reading data from Hopsworks, using ArrowFlight (0.97s) 


In [37]:
iris_fg = fs.get_feature_group(name="el", version=1)
df = iris_fg.read()
# Assuming 'df' is your DataFrame
sorted_df = df.sort_values(by='time')
sorted_df

Finished: Reading data from Hopsworks, using ArrowFlight (1.10s) 


,time,battery,temperature,x,y,z,id,rssi,snr,package_type
891,2024-04-30 00:08:29.799000,NaN,10.125,310,-485,-761,0,-79,8.8,PackageType.HEART_BEAT
120,2024-04-30 00:18:27.799000,NaN,10.125,306,-485,-758,1,-79,8.8,PackageType.HEART_BEAT
235,2024-04-30 00:28:27.799000,3.18,10.000,308,-484,-760,2,-79,8.8,PackageType.HEART_BEAT
467,2024-04-30 00:38:25.970000,NaN,10.000,312,-484,-765,3,-80,9.2,PackageType.HEART_BEAT
576,2024-04-30 00:48:25.970000,NaN,9.875,309,-484,-764,4,-80,9.2,PackageType.HEART_BEAT
...,...,...,...,...,...,...,...,...,...,...
39,2024-05-06 23:14:38.533000,NaN,11.375,298,-484,-764,947,-73,8.8,PackageType.HEART_BEAT
349,2024-05-06 23:24:39.533000,NaN,11.375,297,-486,-766,948,-73,8.8,PackageType.HEART_BEAT
427,2024-05-06 23:34:38.533000,3.20,11.375,297,-484,-765,949,-73,8.8,PackageType.HEART_BEAT
870,2024-05-06 23:44:38.314000,NaN,11.375,299,-485,-764,950,-74,8.5,PackageType.HEART_BEAT


In [31]:
batch_data = feature_view.get_batch_data()
batch_data = batch_data[['x','y','z']]

y_pred = model.predict(batch_data)

y_pred

Finished: Reading data from Hopsworks, using ArrowFlight (0.86s) 


array(['PackageType.HEART_BEAT', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANGE',
       'PackageType.CHANGE', 'PackageType.CHANGE', 'PackageType.CHANG

In [32]:
import pandas as pd
df = pd.DataFrame(y_pred, columns=['prediction'])


In [35]:
df.value_counts()

prediction            
PackageType.CHANGE        943
PackageType.HEART_BEAT      9
Name: count, dtype: int64